In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd "drive/MyDrive/OwnProcedure"

In [ ]:
dataset_length = 1000
path = "./dataset"
generator_path = "./dataset_generator"
items_path = generator_path+"/items"
backgrounds_path = generator_path+"/backgrounds"
crafting_field_coords = [(392, 72), (464, 72), (392, 144), (464, 144)]
prompt = "Pick up the {dragged_obj}."
items = ["oak_planks.png", "grass_block.png", "cobblestone.png"]
items_per_obs = 3
slot_width = 64
slot_height = 64
cursor_x = 352
cursor_y = 332
inv_x = 928
inv_y = 376
output_width = 640
output_height = 360

In [ ]:
import os
import numpy as np
import random
from PIL import Image

In [ ]:
if not os.path.exists(path):
  os.makedirs(path)

In [ ]:
def img_to_mask(img):
  return Image.fromarray(np.uint8((img[:, :, 3] > 0)*255))

In [ ]:
with open(generator_path+"/inventory.txt") as f:
  coords = [(int(x), int(y)) for x, y in [coord.replace("\n", "").split() for coord in f.readlines()]]

imgs = []
masks = []
backgrounds = []
for item in items:
  img = np.array(Image.open(items_path+"/"+item).convert("RGBA").resize((slot_width, slot_height), Image.Resampling.NEAREST))
  mask = img_to_mask(img)
  imgs.append(Image.fromarray(img))
  masks.append(mask)
inventory = Image.open(generator_path+"/inventory.png").convert("RGBA")
inventory_mask = img_to_mask(np.array(inventory))
cursor = Image.open(generator_path+"/cursor.png").convert("RGBA")
cursor_mask = img_to_mask(np.array(cursor))
for background_file in os.listdir(backgrounds_path):
  backgrounds.append(Image.open(backgrounds_path+"/"+background_file).convert("RGB"))

In [ ]:
for i in range(dataset_length):
  if (i+1)%100 == 0:
    print(f"generated batch {i+1-100} - {i+1}")
  folder_path = path+"/"+str(i+1).zfill(12)
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)
  indeces = random.sample(range(items_per_obs), items_per_obs)
  first_index = indeces[0]
  prompt_asset = imgs[first_index]
  prompt_asset_segm = masks[first_index]
  with open(f"{folder_path}/prompt.txt", "w") as f:
    f.write(prompt)
  prompt_asset.save(f"{folder_path}/asset.png")
  prompt_asset_segm.save(f"{folder_path}/asset_segm.png")

  rand_coords = random.sample(coords, items_per_obs)
  inv = inventory.copy()
  for i, (x, y) in enumerate(rand_coords):
    inv.paste(imgs[indeces[i]], (x, y), masks[indeces[i]])
  inv.paste(cursor, (cursor_x, cursor_y), cursor_mask)
  obs = random.sample(backgrounds, 1)[0]
  obs.paste(inv, (inv_x, inv_y), inventory_mask)
  obs = obs.resize((output_width, output_height))
  obs.save(f"{folder_path}/obs.png")

  target_x, target_y = rand_coords[0]
  target_x = (target_x+slot_width/2)/inventory.width*100
  target_y = (target_y+slot_height/2)/inventory.height*100

  with open(f"{folder_path}/actions.txt", "w") as f:
    f.write(f"{target_x} {target_y} {1} {0} {0} {0}")